# **SMS SPAM Classification**

# Import required library

In [1]:
import pandas as pd
import numpy as np
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

# Read dataset

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/spam.csv', delimiter=',', encoding='latin-1')
df.head()


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


# Pre-Processing 

In [37]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [38]:
X = df.v2
Y = df.v1
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)
Y = Y.reshape(-1,1)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [40]:
tokenizer = Tokenizer(num_words=2000, lower=True)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences, maxlen=200)

# Create Model

In [41]:
model  = Sequential()

# Add layers

In [42]:
model.add(Embedding(2000, 50, input_length=200))
model.add(LSTM(64))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1,activation="sigmoid"))

In [43]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 50)           100000    
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 146,337
Trainable params: 146,337
Non-trainable params: 0
_________________________________________________________________


# Compile the Model

In [44]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

# Fit the Model

In [45]:
model.fit(X_train, y_train, batch_size=128, epochs=10, validation_split=0.2)

Epoch 1/10
28/28 [==============================] - 14s 375ms/step - loss: 0.3071 - accuracy: 0.8830 - val_loss: 0.1366 - val_accuracy: 0.9720
Epoch 2/10
28/28 [==============================] - 10s 354ms/step - loss: 0.0757 - accuracy: 0.9815 - val_loss: 0.0677 - val_accuracy: 0.9832
Epoch 3/10
28/28 [==============================] - 10s 356ms/step - loss: 0.0373 - accuracy: 0.9888 - val_loss: 0.0651 - val_accuracy: 0.9809
Epoch 4/10
28/28 [==============================] - 10s 363ms/step - loss: 0.0246 - accuracy: 0.9930 - val_loss: 0.0715 - val_accuracy: 0.9832
Epoch 5/10
28/28 [==============================] - 20s 728ms/step - loss: 0.0151 - accuracy: 0.9950 - val_loss: 0.0763 - val_accuracy: 0.9787
Epoch 6/10
28/28 [==============================] - 12s 427ms/step - loss: 0.0125 - accuracy: 0.9958 - val_loss: 0.0702 - val_accuracy: 0.9821
Epoch 7/10
28/28 [==============================] - 13s 476ms/step - loss: 0.0052 - accuracy: 0.9980 - val_loss: 0.1010 - val_accuracy: 0.9809

# Save the Model

In [47]:
model.save("model.h5")

# Test the Model

In [48]:
test_sequences = tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(test_sequences, maxlen=200)
acc = model.evaluate(X_test, y_test)

35/35 [==============================] - 1s 31ms/step - loss: 0.0621 - accuracy: 0.9892


In [49]:
def predict(message):
    txt = tokenizer.texts_to_sequences(message)
    txt = sequence.pad_sequences(txt, maxlen=200)
    preds = model.predict(txt)
    if preds > 0.5:
        print("Spam")
    else:
        print("Not Spam")

In [50]:
predict(["Sorry, I'll call later"])

1/1 [==============================] - 1s 536ms/step
Not Spam


In [51]:
predict(["Congratulations ur awarded 500 of CD vouchers or 125gift guaranteed & Free entry 2 100 wkly draw txt MUSIC to 87066 TnCs www.Ldew.com1win150ppmx3age16"])

1/1 [==============================] - 0s 27ms/step
Spam
